<a href="https://colab.research.google.com/github/longevity-genie/just-agents/blob/main/examples/notebooks/03_coding_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Just-Agents tutorial

In this tutorial we will develop an LLM agent that can code with just-agents library.

It is the third tutorial in a series of tutorials, other ones are:
1. Basic agents tutorial 
2. Database agent tutorial 
3. Coding agent tutorial  (THIS ONE) 

Here we will write an agent that can write code and install dependencies. The default just-agents code agent uses safe non-root docker sandboxes to isolate environment from the host. However, such approach will not work with google collab, so we will write a coding agent from scratch

Install just-agents library and provide API keys for Groq and OpenAi

In [1]:
!pip install just-agents-core==0.6.1
!pip install just-agents-examples==0.6.1


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import sqlite3

def db_query(sql: str):
    """ This function execute query on open-genes sqlite table. It returns query results. """
    conn = sqlite3.connect("open_genes.sqlite")
    cursor = conn.cursor()

    cursor.execute(sql)
    try:
        rows = cursor.fetchall()
        if rows is None or len(rows) == 0:
            conn.close()
            return ""
        names = [description[0] for description in cursor.description]
        text = "; ".join(names) + "\n"
        for row in rows:
            row = [str(i) for i in row]
            text += "; ".join(row) + "\n"
    finally:
        conn.close()

    return text

### Run functions to run code

Writing custom functions to run code and install dependencies

In [3]:
# Define the function
import io
import contextlib

def run_python_code(code: str):
    """
    code: str # python code to run
    """
    output = io.StringIO()
    error = io.StringIO()

    try:
        with contextlib.redirect_stdout(output), contextlib.redirect_stderr(error):
            exec(code)
        result = output.getvalue()
        if result:
            return result.strip()
        else:
            return "No output."
    except Exception as e:
        return f"Error: {str(e)}"

Let's test that code generation works

In [4]:
run_python_code("print('Hello World')")

'Hello World'

In [7]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent
import pprint

prompt = "Compute 2 * 10 + (342 +3) / 23.0"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.LLAMA3_3,
    tools=[run_python_code]
)
response = agent.query(prompt)
f"""MODEL RESPONSE = "{response}" """
agent.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Compute 2 * 10 + (342 +3) / 23.0                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to run_python_code:                                                                                   │
│ {"code": "print(2 * 10 + (342 + 3) / 23.0)"}                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from run_python_code:                                                                                  │
│ 35.0                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ The result of the computation is 35.0.                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Dependency hell management

But what if it does not have dependencies installed?

In [8]:
agent.query("translate ATGCATAGGTAG with biopython")

'I am not able to execute the biopython code as it seems that the biopython library is not installed in the environment. However, I can tell you that the translation of the DNA sequence "ATGCATAGGTAG" is done by reading the sequence in codons (sequences of three nucleotides) and using the standard genetic code to determine the corresponding amino acid for each codon.\n\nHere is the correct translation:\n\nATG -> M (Methionine)\nCAT -> H (Histidine)\nAGG -> R (Arginine)\nTAG -> * (Stop codon)\n\nSo, the translation of the DNA sequence "ATGCATAGGTAG" is M-H-R*.'

In [9]:
import subprocess

def execute_bash(command: str):
    """
    Executes a Bash command and returns the output or error message.

    Parameters:
    command (st): Bash command as a string.

    Returns:
    str: The output or error message from executing the command.
    """
    try:
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        if result.returncode == 0:
            return result.stdout.strip() if result.stdout else "No output."
        else:
            return f"Error: {result.stderr.strip()}"
    except Exception as e:
        return f"Exception: {str(e)}"

# Example usage:
bash_command = "ls -la"
output = execute_bash(bash_command)
print(output)


total 2692
drwxrwxr-x 2 antonkulaga antonkulaga    4096 Mar  1 20:25 .
drwxrwxr-x 5 antonkulaga antonkulaga    4096 Mar  1 17:37 ..
-rw-rw-r-- 1 antonkulaga antonkulaga  592177 Mar  1 20:19 01_just_agents_colab.ipynb
-rw-rw-r-- 1 antonkulaga antonkulaga  218738 Mar  1 20:26 02_sqlite_example.ipynb
-rw-rw-r-- 1 antonkulaga antonkulaga   84337 Mar  1 20:27 03_coding_agent.ipynb
-rw-rw-r-- 1 antonkulaga antonkulaga    1687 Mar  1 20:12 agent_profiles.yaml
-rw-rw-r-- 1 antonkulaga antonkulaga    1259 Mar  1 20:02 .env
-rw-r--r-- 1 antonkulaga antonkulaga 1150976 Mar  1 20:22 open_genes.sqlite
-rw-rw-r-- 1 antonkulaga antonkulaga  684578 Mar  1 20:25 open_genes.txt


In [10]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent
import pprint


agent: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4_1NANO,
    tools=[run_python_code, execute_bash]
)
agent.memory.add_on_message(lambda m: pprint.pprint(m))

result = agent.query("Install biopython with pip")
result

{'content': 'Install biopython with pip', 'role': user}
{'content': 'You are a helpful AI assistant', 'role': system}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"command":"pip install '
                                           'biopython"}',
                              'name': 'execute_bash'},
                 'id': 'call_Pw3Z9P7NM88QsjW0hB8aIuFR',
                 'type': 'function'}]}
{'content': 'Collecting biopython\n'
            '  Downloading '
            'biopython-1.85-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata '
            '(13 kB)\n'
            'Collecting numpy (from biopython)\n'
            '  Downloading '
            'numpy-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata '
            '(62 kB)\n'
            'Downloading '
            'biopython-1.85-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl '
            '(3.3 MB)\n'
            '\x1b[?25l   '
            '\x1b━━━━━━━━━━━

'Biopython has been successfully installed along with its dependency, NumPy. If you need any further assistance or have questions about using Biopython, feel free to ask!'

In [11]:
agent.query("translate ATGCATAGGTAG with biopython")
agent.memory.pretty_print_all_messages()

{'content': 'translate ATGCATAGGTAG with biopython', 'role': user}
{'content': 'You are a helpful AI assistant', 'role': system}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"code":"from Bio.Seq import '
                                           'Seq\\n\\n# Create a sequence '
                                           'object\\nsequence = '
                                           "Seq('ATGCATAGGTAG')\\n\\n# "
                                           'Translate the '
                                           'sequence\\ntranslated_sequence = '
                                           'sequence.translate()\\ntranslated_sequence"}',
                              'name': 'run_python_code'},
                 'id': 'call_2MqSJF8BHQwaeBZPRXQ4YRHb',
                 'type': 'function'}]}
{'content': 'No output.',
 'name': 'run_python_code',
 'role': 'tool',
 'tool_call_id': 'call_2MqSJF8BHQwaeBZPRXQ4YRHb'}
{'content': 'It seems that the translation did not produ

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Install biopython with pip                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to execute_bash:                                                                                      │
│ {"command":"pip install biopython"}                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from execute_bash:                                                                                     │
│ Collecting biopython                                                                                            │
│   Downloading biopython-1.85-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)        │
│ Collecting numpy (from biopython)                                                                               │
│   Downloading numpy-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)           │
│ Downloading biopython-1.85-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)                  │
│    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.3 MB ? eta -:--:--     │
│    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.5 MB/s eta       │
│ 0:00:00                                                                                                  │
│ Downloading numpy-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)               │
│    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.1 MB ? eta -:--:--    │
│    ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 6.6/16.1 MB │
│ 32.6 MB/s eta 0:00:01                                                                             │
│    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━             │
│ 13.1/16.1 MB 32.6 MB/s eta 0:00:01                                                         │
│    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 30.7 MB/s eta     │
│ 0:00:00                                                                                                  │
│ Installing collected packages: numpy, biopython                                                            │
│ Successfully installed biopython-1.85 numpy-2.2.3                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to run_python_code:                                                                                   │
│ {"code":"from Bio.Seq import Seq\n\n# Create a sequence object\nsequence = Seq('ATGCATAGGTAG')\n\n# Translate   │
│ the sequence\ntranslated_sequence = sequence.translate()\n\n# Output the translated                             │
│ sequence\nstr(translated_sequence)"}                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from run_python_code:                                                                                  │
│ No output.                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to run_python_code:                                                                                   │
│ {"code":"from Bio.Seq import Seq\n\n# Create a sequence object\nsequence = Seq('ATGCATAGGTAG')\n\n# Check the   │
│ length of the sequence\nlength = len(sequence)\n\n# Translate the sequence\ntranslated_sequence =               │
│ sequence.translate()\n\nlength, str(translated_sequence)"}                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from run_python_code:                                                                                  │
│ No output.                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ It seems that the code execution is not returning any output. However, I can explain how the translation works. │
│                                                                                                                 │
│ The sequence "ATGCATAGGTAG" is a DNA sequence. When translated, it converts the nucleotide sequence into an     │
│ amino acid sequence based on the genetic code. The translation process requires the sequence to be in multiples │
│ of three (codons).                                                                                              │
│                                                                                                                 │
│ 1. **Length Check**: The sequence length is 12, which is a multiple of 3, so it should be translatable.         │
│ 2. **Translation**: The translation of the sequence will yield a corresponding amino acid sequence.             │
│                                                                                                                 │
│ If you have access to a Python environment with Biopython installed, you can run the following code:            │
│                                                                                                                 │
│ ```python                                                                                                       │
│ from Bio.Seq import Seq                                                                                         │
│                                                                                                                 │
│ # Create a sequence object                                                                                      │
│ sequence = Seq('ATGCATAGGTAG')                                                                                  │
│                                                                                                                 │
│ # Translate the sequence                                                                                        │
│ translated_sequence = sequence.translate()                                                                      │
│                                                                                                                 │
│ # Output the translated sequence                                                                                │
│ print(translated_sequence)                                                                                      │
│ ```                                                                                                             │
│                                                                                                                 │
│ This should give you the translated amino acid sequence. If you need further assistance or want me to try       │
│ something else, please let me know!                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯